In [32]:
%reload_ext autoreload
%autoreload 2
import os
import json
import pickle as pkl
import collections
import numpy as np
import pandas as pd
import sklearn.metrics
from modeling import get_model_cls
from analysis_util import read_file, load_all, get_best_trial
from encode_data import Encoder, Mapping

In [33]:
import tensorflow as tf
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [34]:
output_dir = 'recommend_V2_NN/outputs/nn_outputs'
data_file = 'recommend_V2_NN/data/raw_data/Hackathon_data_JanandFeb.csv'
df = read_file(data_file)

In [35]:
best_trial = get_best_trial(output_dir)

best metric: 0.009605477564036846, best_trial: recommend_V2_NN/outputs/nn_outputs/model_3


In [40]:
model_config_file = os.path.join('recommend_V2_NN/outputs/nn_outputs/model_3',
#                                  best_trial,
                                 'model_config.json').replace("\\","/") #required for windows machine
with open(model_config_file, 'r') as f:
    model_config = json.load(f)

In [41]:
encoder_path = 'recommend_V2_NN/data/encoded_data/encoder.pkl'
file = open(encoder_path, 'rb')

In [42]:
encoder = pkl.load(file)

In [44]:
model = get_model_cls(model_config['model_type'])(encoder.text_config, model_config)
# model.load(best_trial)
model.load('recommend_V2_NN/outputs/nn_outputs/model_3')

2022-04-19 22:48:26.658016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-19 22:48:26.661468: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-19 22:48:26.664760: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-AD7TO7E): /proc/driver/nvidia/version does not exist
2022-04-19 22:48:26.682627: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [45]:
y, X, _ = encoder.transform(df)

Starting to encode dataset...
Starting to encode structural data...
Except boolean, categorical and text input data, the X shape is (1203, 16)
we have normalized 16 columns: ['Age', 'TotalUsers', 'TotalUsersWithSkuAssigned', 'SubscriptionsCount', 'EXOEnabledUsers', 'SPOEnabledUsers', 'SFBEnabledUsers', 'OD4BEnabledUsers', 'TeamEnabledUsers', 'PaidSeats', 'PaidEXOSeats', 'PaidOD4BSeats', 'PaidProplusSeats', 'PaidSPOSeats', 'PaidTeamsSeats', 'PaidSFBSeats'].
Non-text input data after encoding, the shape is (1203, 269)
We have 269 columns.
complete encoding part of structural data!
complete encoding part of textual data!


In [47]:
# %time pred = model.predict(X, output_dir=best_trial)
%time pred = model.predict(X)

CPU times: user 109 ms, sys: 93.8 ms, total: 203 ms
Wall time: 133 ms


In [49]:
path_to_save = os.path.join('recommend_V2_NN/outputs/', 'prediction.npy')
np.save(path_to_save, pred)

In [11]:
mse = sklearn.metrics.mean_squared_error(y, pred)
mse

0.028676112490326793

In [12]:
model.model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 structual_data (InputLayer)  [(None, 269)]            0         
                                                                 
 dense_14 (Dense)            (None, 48)                12960     
                                                                 
 dense_15 (Dense)            (None, 48)                2352      
                                                                 
 dense_16 (Dense)            (None, 30)                1470      
                                                                 
 dense_17 (Dense)            (None, 30)                930       
                                                                 
 dense_18 (Dense)            (None, 8)                 248       
                                                                 
Total params: 17,960
Trainable params: 17,960
Non-trainable

## Generate prediction results table


In [13]:
pred

array([[ 0.58349025,  0.00180508, -0.00200222, ...,  0.81120396,
         0.35594106,  0.6604699 ],
       [ 0.52957183,  0.15545826, -0.00549661, ...,  0.93781674,
         0.75286543,  0.76701367],
       [ 0.17081356, -0.0142157 , -0.00290386, ...,  0.27520618,
         0.75183576,  0.4805825 ],
       ...,
       [ 0.2802615 ,  0.01078228,  0.93976665, ...,  0.8571981 ,
         0.9241239 ,  0.6799935 ],
       [ 0.44014445,  0.18876457,  0.609166  , ...,  0.0699264 ,
         0.8602202 ,  0.22869389],
       [ 1.0378836 ,  0.9306137 ,  0.8792633 , ...,  0.43960828,
         0.9003432 ,  0.90925205]], dtype=float32)

In [14]:
metadata_file = 'recommend_V2_NN/data/raw_data/metadata.json'
with open(metadata_file, 'r') as f:
    metadata = json.load(f)

In [32]:
model_config['task_type']

'regression'

In [15]:
output_col = metadata['output_label']

In [25]:
pred_col = []
for c in output_col:
    w = c + '_pred'
    pred_col.append(w)
pred_col

['PowerPoint_MAR_pred',
 'OneNote_MAR_pred',
 'Outlook_MAR_pred',
 'EXO_MAR_pred',
 'OD4B_MAR_pred',
 'SPO_MAR_pred',
 'SkypeTeams_MAR_pred',
 'WordExcel_MAR_pred']

In [27]:
df.head()

,TenantId,Age,Age Group,AreaName,CountryCode,IndustryName,TotalUsers,TotalUsersWithSkuAssigned,SubscriptionsCount,EXOEnabledUsers,SPOEnabledUsers,SFBEnabledUsers,OD4BEnabledUsers,TeamEnabledUsers,PaidSeats,PaidEXOSeats,PaidOD4BSeats,PaidProplusSeats,PaidSPOSeats,PaidTeamsSeats,PaidSFBSeats,PowerPoint_MAR_Rank,OneNote_MAR_Rank,Outlook_MAR_Rank,EXO_MAR_Rank,OD4B_MAR_Rank,SPO_MAR_Rank,SkypeTeams_MAR_Rank,WordExcel_MAR_Rank,PowerPoint_MAR,OneNote_MAR,Outlook_MAR,EXO_MAR,OD4B_MAR,SPO_MAR,SkypeTeams_MAR,WordExcel_MAR,Excel_AllUp,Word_AllUp,PowerPoint_AllUp,OneNote_AllUp,Outlook_AllUp,EXO_AllUp,SPO_AllUp,OD4B_AllUp,Teams_AllUp,SfB_AllUp,SkypeTeams_AllUp,WordExcel_AllUp
0,a1f51839-24c4-428b-a708-0019dc6842ee,2863,<10yr,United States,USA,Media & Entertainment,85,8,5,19,18,18,18,18,19.0,19.0,19.0,19.0,19.0,19.0,19.0,2,4,4,1,3,1,3,2,0.63,0.00,0.0,0.84,0.52,0.84,0.52,0.63,11.0,12.0,12.0,0.0,0.0,16.0,16.0,10.0,10.0,0.0,10.0,12.0
1,e441e2e0-ef38-469a-a0d5-004701ccaf9b,2672,<10yr,MEA,ISR,Other - Unsegmented,38,18,6,18,19,18,19,19,16.0,16.0,16.0,14.0,16.0,16.0,16.0,4,6,7,1,5,1,3,2,0.71,0.07,0.0,1.00,0.56,1.00,0.87,0.92,12.0,13.0,10.0,1.0,0.0,16.0,17.0,9.0,14.0,0.0,14.0,13.0
2,30edaab1-2f20-4e20-b98b-0088595779ca,1363,<10yr,United States,USA,Others,16,13,5,12,11,11,12,11,12.0,12.0,12.0,11.0,11.0,11.0,11.0,6,7,7,1,3,5,2,4,0.18,0.00,0.0,1.00,0.75,0.27,0.81,0.72,8.0,8.0,2.0,0.0,0.0,13.0,3.0,9.0,9.0,0.0,9.0,8.0
3,7433f7ec-d409-4daa-82cb-00cea0acf72d,315,<1yr,Western Europe,NLD,Others,18,17,5,17,11,10,11,11,17.0,17.0,9.0,9.0,9.0,9.0,9.0,1,2,3,1,1,1,1,1,1.00,0.55,0.0,1.00,1.00,1.00,1.00,1.00,13.0,12.0,10.0,5.0,0.0,17.0,14.0,11.0,10.0,0.0,10.0,13.0
4,1aa5a1d5-30d7-4728-ba6f-0121b75e7a68,536,<3yr,India,IND,Others,457,99,5,97,101,86,101,101,86.0,86.0,86.0,86.0,86.0,86.0,86.0,2,4,5,1,2,1,1,3,0.98,0.55,0.0,1.00,0.98,1.00,1.00,0.94,81.0,79.0,85.0,48.0,0.0,90.0,193.0,85.0,131.0,3.0,131.0,81.0


In [30]:
results_df = pd.DataFrame(data=pred, index=df.TenantId, columns=pred_col)
results_df = results_df.reset_index()
for c in output_col:
    results_df[c] = df[c]
results_df.head()

,TenantId,PowerPoint_MAR_pred,OneNote_MAR_pred,Outlook_MAR_pred,EXO_MAR_pred,OD4B_MAR_pred,SPO_MAR_pred,SkypeTeams_MAR_pred,WordExcel_MAR_pred,PowerPoint_MAR,OneNote_MAR,Outlook_MAR,EXO_MAR,OD4B_MAR,SPO_MAR,SkypeTeams_MAR,WordExcel_MAR
0,a1f51839-24c4-428b-a708-0019dc6842ee,0.583490,0.001805,-0.002002,0.911549,0.255900,0.811204,0.355941,0.660470,0.63,0.00,0.0,0.84,0.52,0.84,0.52,0.63
1,e441e2e0-ef38-469a-a0d5-004701ccaf9b,0.529572,0.155458,-0.005497,0.991763,0.393121,0.937817,0.752865,0.767014,0.71,0.07,0.0,1.00,0.56,1.00,0.87,0.92
2,30edaab1-2f20-4e20-b98b-0088595779ca,0.170814,-0.014216,-0.002904,0.870951,0.621325,0.275206,0.751836,0.480583,0.18,0.00,0.0,1.00,0.75,0.27,0.81,0.72
3,7433f7ec-d409-4daa-82cb-00cea0acf72d,0.698970,0.262255,0.003674,0.782171,0.801193,0.790579,0.904125,0.708147,1.00,0.55,0.0,1.00,1.00,1.00,1.00,1.00
4,1aa5a1d5-30d7-4728-ba6f-0121b75e7a68,0.475080,0.089159,0.000148,0.932760,0.567667,0.844569,0.936444,0.663068,0.98,0.55,0.0,1.00,0.98,1.00,1.00,0.94


In [34]:
# results_df.to_csv('results.csv', index=None)

## Extract Features from the n layers of a MLP

In [8]:
from keras import backend as K 
inp = model.model.input
out = model.model.layers[4].output
functors = K.function([inp], [out]) 

layer_outs = functors([X])
   
print(layer_outs)

In [17]:
arr = layer_outs[0]
arr.shape

(1203, 30)

In [ ]:
def read_file(path):
    filename, file_extension = os.path.splitext(path)
    if file_extension == '.csv':
        sep = ','
    elif file_extension == '.tsv':
        sep = '\t'
    else:
        raise ValueError('Unknown type of file: {}. Please add .csv or .tsv.'.format(path))
    df = pd.read_csv(path, sep=sep)
    return df


## use dict like object
class Mapping(dict):

    def __getattr__(self, name):
        if name in self:
            return self[name]
        else:
            raise AttributeError("No such attribute: " + name)

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        if name in self:
            del self[name]
        else:
            raise AttributeError("No such attribute: " + name)


def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')


def contain_nontext_features(metadata):
    n_dtype = len(metadata.keys())

    if n_dtype == 1 and 'input_text' in metadata.keys():
        return False
    else:
        return True


def separate_input_output_cols(df, metadata):
    """According to the metadata, separate the input features, output features and
        different types of input features.

    Args:
      df: a DataFrame that stores the raw data.
      metadata: a dictionary that stores the detail description for features.
        metadata = {
        'output_type': 'classes' for classification task (or it can be 'numbers' for regression task)
        'input_features': ['TenantId','CreatedDate', ...],
        'output_label': ['AR_exchange_06','AR_sharepoint_06', ...],
        'input_bool': ['HasEXO','HasSPO', ...],
        'input_categorical': ['CountryCode', 'Languange', ...],
        'input_datetime': ['CreatedDate', ...],
        'input_int': [...] ,
        'input_float': [...]
        }
    Returns:
      df_y: a DataFrame that stores the output labels
      df_X_text: a DataFrame that stores the textual input
      df_X_float: a DataFrame that stores the float inputs
      df_X_int: a DataFrame that stores the integer inputs
      df_X_cat: a DataFrame that stores the categorical inputs
      df_X_datetime: a DataFrame that stores the datetime inputs
      df_X_bool: a DataFrame that stores the boolean inputs

    """
    # input_cols = metadata['input_features']
    output_cols = metadata['output_label']
    input_text_cols = metadata['input_text']
    input_float_cols = metadata['input_float']
    input_int_cols = metadata['input_int']
    input_cat_cols = metadata['input_categorical']
    input_datetime_cols = metadata['input_datetime']
    input_bool_cols = metadata['input_bool']

    df_y = df.loc[:, output_cols]
    df_X_text = df.loc[:, input_text_cols]
    df_X_float = df.loc[:, input_float_cols]
    df_X_int = df.loc[:, input_int_cols]
    df_X_cat = df.loc[:, input_cat_cols]
    df_X_datetime = df.loc[:, input_datetime_cols]
    df_X_bool = df.loc[:, input_bool_cols]

    return df_y, df_X_text, df_X_float, df_X_int, df_X_cat, df_X_datetime, df_X_bool


def encode_datetime(df_X_datetime):
    """Encode the datetime inputs from '2/5/2014 5:31:19 AM' format
        to a numerical number of UTC format.

    Args:
      df_: a DataFrame that only stores the datetime inputs.

    Returns:
      X_datetime: a numpy array that contains the encoded datetime inputs.
      datetime_cols: a list that contains the datetime colunms name.

    """

    cols = df_X_datetime.columns
    for i in cols:
        df_X_datetime[i] = pd.to_datetime(df_X_datetime[i], utc=True,
                                          errors='coerce').astype(int, errors='ignore')

    X_datetime = df_X_datetime.to_numpy()

    return X_datetime


def encode_bool(df_X_bool):
    """Encode the numerical and boolean inputs.

    Args:
      df_X_bool: a DataFrame that stores the boolean inputs

    Returns:
      X_bool: a numpy array that contains the encoded boolean inputs.

    """
    X_bool = df_X_bool.astype(int).to_numpy()
    return X_bool


def encode_num(df_X_num):
    """Encode the numerical and boolean inputs.

    Args:
      df_X_num: a DataFrame that stores the numerical inputs

    Returns:
      X_num: a numpy array that contains the float inputs.

    """
    X_num = df_X_num.to_numpy()
    return X_num


def encode_y(metadata, df_y, y_encoder):
    if metadata['output_type'] == 'classes':
        # encode class values as integers
        y_arr = df_y.values
        if y_encoder is None:
            y_encoder = LabelEncoder()
            y = y_encoder.fit_transform(y_arr)
        else:
            y = y_encoder.transform(y_arr)

        if len(y_encoder.classes_) > 2:
            # convert integers to dummy variables (i.e. one hot encoded)
            y = np_utils.to_categorical(y)

    elif metadata['output_type'] == 'numbers':
        y = df_y.to_numpy()
        y_encoder = None

    else:
        raise ValueError('Unknown type of output: {}'.format(metadata['output_type']))

    return y, y_encoder


def encode_strucdata(metadata, df_X_float, df_X_int, df_X_cat, df_X_datetime, df_X_bool, vectorizer, scaler):
    """Encode the meta data part in dataset, such as numerical and categorical data.

    """
    print('Starting to encode structural data...')

    # df_y, _, df_X_float, df_X_int, df_X_cat, df_X_datetime, df_X_bool = separate_input_output_cols(df, metadata)

    X_list = []
    cols_name = []

    if df_X_float.shape[1] > 0:
        X_float = encode_num(df_X_float)
        X_list.append(X_float)
        cols_name += metadata['input_float']

    if df_X_int.shape[1] > 0:
        X_int = encode_num(df_X_int)
        X_list.append(X_int)
        cols_name += metadata['input_int']

    if df_X_datetime.shape[1] > 0:
        X_datetime = encode_datetime(df_X_datetime)
        X_list.append(X_datetime)
        cols_name += metadata['input_datetime']

    if X_list:
        ### normalize all the inputs except boolean, categorical, and text features
        X_arr = np.concatenate(X_list, axis=1)

        if scaler == None:
            scaler = StandardScaler()
            X_struc = scaler.fit_transform(X_arr)
        else:
            X_struc = scaler.transform(X_arr)
        assert len(cols_name) == X_struc.shape[1]
        print('Except boolean, categorical and text input data after encoding, the shape is {}'.format(X_struc.shape))
        print('we have {} columns.'.format(len(cols_name)))
    else:
        X_struc = None

    ### encode boolean columns
    if df_X_bool.shape[1] > 0:
        X_bool = encode_bool(df_X_bool)
        cols_name += metadata['input_bool']
        if X_struc is None:
            X_struc = X_bool
        else:
            X_struc = np.concatenate([X_struc, X_bool], axis=1)

    ### encode the categorical columns
    if df_X_cat.shape[1] > 0:
        X_cat_dict = df_X_cat.to_dict(orient='records')

        if vectorizer == None:
            vectorizer = DictVectorizer(sparse=False)
            X_cat = vectorizer.fit_transform(X_cat_dict)

        else:
            X_cat = vectorizer.transform(X_cat_dict)

        vocab = vectorizer.vocabulary_
        vocab_od = collections.OrderedDict(sorted(vocab.items(), key=lambda x: x[1]))
        cat_encoded_cols = list(vocab_od.keys())
        cols_name += cat_encoded_cols
        if X_struc is None:
            X_struc = X_cat
        else:
            X_struc = np.concatenate([X_struc, X_cat], axis=1)

    assert len(cols_name) == X_struc.shape[1]
    print('Non-text input data after encoding, the shape is {}'.format(X_struc.shape))
    print('We have {} columns.'.format(len(cols_name)))

    return X_struc, vectorizer, scaler


def open_glove(glove_file_path):
    print('Indexing word vectors.')

    embeddings_index = {}
    f = open(glove_file_path, encoding="utf8")
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))

    return embeddings_index


def encode_textdata(df_X_text, tokenizer, mode, max_words, maxlen):
    ## encode text columns, encoded text features should not be normalized.

    print('Starting to encode text inputs...')

    texts = df_X_text.iloc[:, 0].values.astype('U')
    print('Found %s texts.' % len(texts))

    if mode == 'tfidf':
        if tokenizer is None:
            tokenizer = Tokenizer(num_words=max_words)
            tokenizer.fit_on_texts(texts)
        X_text = tokenizer.texts_to_matrix(texts, mode='tfidf')
        print('tfidf X_text shape: {}'.format(X_text.shape))

    elif mode == 'glove':
        # vectorize the text samples into a 2D integer tensor
        if tokenizer is None:
            tokenizer = Tokenizer(num_words=max_words, oov_token='<UNK>')
            tokenizer.fit_on_texts(texts)
            tokenizer.word_index = {e: i for e, i in tokenizer.word_index.items() if i <= max_words}
            # tokenizer.word_index[tokenizer.oov_token] = max_words + 1

        sequences = tokenizer.texts_to_sequences(texts)

        word_index = tokenizer.word_index
        print('Found %s unique tokens.' % len(word_index))

        X_text = pad_sequences(sequences, maxlen=maxlen, padding='post')
    else:
        raise ValueError('Unknown text processing mode: {}'.format(mode))

    return X_text, tokenizer  ### need to save embedding_matrix as well


def encode_dataset(df, metadata, y_encoder=None, vectorizer=None, scaler=None, tokenizer=None, mode=None,
                   max_words=None, maxlen=None):
    print('Starting to encode dataset...')

    df_y, df_X_text, df_X_float, df_X_int, df_X_cat, df_X_datetime, df_X_bool = separate_input_output_cols(df, metadata)

    y, y_encoder = encode_y(metadata, df_y, y_encoder)

    # check if exist non-text data
    if df_X_float.shape[1] + df_X_int.shape[1] + df_X_cat.shape[1] + df_X_datetime.shape[1] + df_X_bool.shape[1] > 0:
        X_struc, vectorizer, scaler = encode_strucdata(metadata, df_X_float, df_X_int, df_X_cat, df_X_datetime,
                                                       df_X_bool, vectorizer, scaler)
    else:
        X_struc, vectorizer, scaler = None, None, None

    print("complete encoding part of structural data!")

    if not metadata['input_text'] or mode == None:
        X_text, tokenizer = None, None
    else:
        X_text, tokenizer = encode_textdata(df_X_text, tokenizer, mode, max_words, maxlen)

    print("complete encoding part of textual data!")
    return y, y_encoder, X_struc, X_text, vectorizer, scaler, tokenizer


class Encoder(object):

    def __init__(self, metadata, text_config):
        self.text_config = text_config
        self.metadata = metadata
        self.has_nontext = contain_nontext_features(metadata)

    def fit_transform(self, df):
        if self.has_nontext and self.text_config is None:
            y, self.y_encoder, X_struc, X_text, self.vectorizer, self.scaler, _ = encode_dataset(df, self.metadata,
                                                                                                 mode=None)

        elif self.text_config.mode == 'tfidf':
            y, self.y_encoder, X_struc, X_text, self.vectorizer, self.scaler, self.tokenizer = encode_dataset(
                df, self.metadata, mode='tfidf', max_words=self.text_config.max_words)

        elif self.text_config.mode == 'glove':
            y, self.y_encoder, X_struc, X_text, self.vectorizer, self.scaler, self.tokenizer = encode_dataset(
                df, self.metadata, mode='glove', max_words=self.text_config.max_words, maxlen=self.text_config.maxlen)

            word_index = self.tokenizer.word_index
            # prepare embedding matrix
            embedding_matrix = np.zeros((len(word_index) + 1, self.text_config.embedding_dim))
            for word, i in word_index.items():
                embedding_vector = self.text_config.embeddings_index.get(word)
                if embedding_vector is not None:
                    # words not found in embedding index will be all-zeros.
                    embedding_matrix[i] = embedding_vector
            self.text_config.embedding_matrix = embedding_matrix
            del self.text_config.embeddings_index

        else:
            raise ValueError('Unknown type of text_config: {}'.format(self.text_config.mode))

        return y, X_struc, X_text

    def transform(self, df):
        if self.text_config is None:
            y, _, X_struc, X_text, _, _, _ = encode_dataset(df, self.metadata, y_encoder=self.y_encoder,
                                                            vectorizer=self.vectorizer, scaler=self.scaler)

        elif self.text_config.mode == 'tfidf':

            y, _, X_struc, X_text, _, _, _ = encode_dataset(
                df, self.metadata, y_encoder=self.y_encoder,
                vectorizer=self.vectorizer, scaler=self.scaler, tokenizer=self.tokenizer, mode='tfidf',
                max_words=self.text_config.max_words)
        elif self.text_config.mode == 'glove':
            y, _, X_struc, X_text, _, _, _ = encode_dataset(
                df, self.metadata, y_encoder=self.y_encoder,
                vectorizer=self.vectorizer, scaler=self.scaler, tokenizer=self.tokenizer,
                mode='glove', max_words=self.text_config.max_words, maxlen=self.text_config.maxlen)
        else:
            raise ValueError('Unknown type of text_config: {}'.format(self.text_config.mode))

        return y, X_struc, X_text

In [18]:
np.save('recommend_V2_NN/data/raw_data/Embedding.npy', arr)

In [19]:
df_id = df['TenantId']
df_id.to_csv('recommend_V2_NN/data/raw_data/Embedding_id.csv', index=None)

In [43]:
inp = model.model.input                                           # input placeholder
outputs = [layer.output for layer in model.model.layers]          # all layer outputs
functors = [K.function([inp], [out]) for out in outputs]    # evaluation functions
# functor = K.function([inp, K.learning_phase()], outputs)   # evaluation function for GPU?

# Testing
# test = np.random.random((269))[np.newaxis,...]
layer_outs = [func([X]) for func in functors]

print(layer_outs)

[[array([[ 5.032131  , -0.3668098 , -0.649486  , ...,  4.        ,
         4.        ,  1.        ],
       [ 0.41434795,  3.4243884 ,  3.9882743 , ...,  1.        ,
         1.        ,  1.        ],
       [-1.8065605 , -0.39763254, -0.75488967, ...,  4.        ,
         3.        ,  1.        ],
       ...,
       [ 0.05708297, 11.592417  , 18.270468  , ...,  1.        ,
         1.        ,  1.        ],
       [ 0.4303449 ,  4.256603  ,  8.994947  , ...,  5.        ,
         2.        ,  3.        ],
       [ 2.3686407 ,  4.179546  ,  5.7801366 , ...,  4.        ,
         5.        ,  3.        ]], dtype=float32)], [array([[0.22717221, 0.        , 1.0796607 , ..., 0.        , 1.6200686 ,
        0.773795  ],
       [1.7365007 , 0.        , 0.8142006 , ..., 2.155461  , 0.        ,
        1.0966704 ],
       [1.2287397 , 0.        , 1.2940555 , ..., 0.        , 0.78642535,
        0.        ],
       ...,
       [2.6641848 , 0.        , 0.5201439 , ..., 3.0237014 , 0.        ,


In [32]:
inp

<KerasTensor: shape=(None, 269) dtype=float32 (created by layer 'structual_data')>

In [34]:
outputs

[<KerasTensor: shape=(None, 269) dtype=float32 (created by layer 'structual_data')>,
 <KerasTensor: shape=(None, 48) dtype=float32 (created by layer 'dense_14')>,
 <KerasTensor: shape=(None, 48) dtype=float32 (created by layer 'dense_15')>,
 <KerasTensor: shape=(None, 30) dtype=float32 (created by layer 'dense_16')>,
 <KerasTensor: shape=(None, 30) dtype=float32 (created by layer 'dense_17')>,
 <KerasTensor: shape=(None, 8) dtype=float32 (created by layer 'dense_18')>]

In [21]:
np.exp(np.log(30))

30.000000000000004

In [22]:
np.random.uniform(30,30)

30.0